In [1]:
import pandas as pd 
import numpy as np 
import math

import pickle

In [2]:
# various constraints on parameters and outputs
MIN_HALF_LIFE = 15.0 / (24 * 60)    # 15 minutes
MAX_HALF_LIFE = 274.                # 9 months
LN2 = math.log(2.)
prob_ceil = .9999
prob_floor = 0.0001


def hclip(h):
    # bound min/max half-life
    return min(max(h, MIN_HALF_LIFE), MAX_HALF_LIFE)

In [3]:
learn_df = pd.read_csv('../data/settles.acl16.learning_traces.13m.csv.gz')

In [4]:
learn_df['p_recall_clip'] = learn_df.p_recall.clip(lower =prob_floor, upper= prob_ceil)
learn_df['delta_days'] = (learn_df.delta/(60*60*24)).astype('float')
learn_df['log_p_recall_clip'] = learn_df.p_recall_clip.apply(math.log,args=([2]))
learn_df['half_life'] = (-learn_df.delta_days/learn_df.log_p_recall_clip).clip(lower= MIN_HALF_LIFE, upper= MAX_HALF_LIFE)
learn_df['wrong'] = learn_df.history_seen - learn_df.history_correct
learn_df['session_wrong'] = learn_df.session_seen = learn_df.session_correct
learn_df['right_sqrt'] = learn_df.history_correct.apply(lambda x: math.sqrt(1+x))
learn_df['wrong_sqrt'] = learn_df.wrong.apply(lambda x: math.sqrt(1+x))
learn_df['bias'] = 1
learn_df['a'] = (learn_df.history_correct + 2.) / (learn_df.history_seen+ 4.)

In [5]:
learn_df.head()

,p_recall,timestamp,delta,user_id,learning_language,ui_language,lexeme_id,lexeme_string,history_seen,history_correct,...,p_recall_clip,delta_days,log_p_recall_clip,half_life,wrong,session_wrong,right_sqrt,wrong_sqrt,bias,a
0,1.0,1362076081,27649635,u:FO,de,en,76390c1350a8dac31186187e2fe1e178,lernt/lernen<vblex><pri><p3><sg>,6,4,...,0.9999,320.018924,-0.000144,274.0,2,2,2.236068,1.732051,1,0.600000
1,0.5,1362076081,27649635,u:FO,de,en,7dfd7086f3671685e2cf1c1da72796d7,die/die<det><def><f><sg><nom>,4,4,...,0.5000,320.018924,-1.000000,274.0,0,1,2.236068,1.000000,1,0.750000
2,1.0,1362076081,27649635,u:FO,de,en,35a54c25a2cda8127343f6a82e6f6b7d,mann/mann<n><m><sg><nom>,5,4,...,0.9999,320.018924,-0.000144,274.0,1,1,2.236068,1.414214,1,0.666667
3,0.5,1362076081,27649635,u:FO,de,en,0cf63ffe3dda158bc3dbd55682b355ae,frau/frau<n><f><sg><nom>,6,5,...,0.5000,320.018924,-1.000000,274.0,1,1,2.449490,1.414214,1,0.700000
4,1.0,1362076081,27649635,u:FO,de,en,84920990d78044db53c1b012f5bf9ab5,das/das<det><def><nt><sg><nom>,4,4,...,0.9999,320.018924,-0.000144,274.0,0,1,2.236068,1.000000,1,0.750000


In [6]:
learn_df['this'] = 

In [9]:
split_point = int(.9 * learn_df.shape[0])

In [10]:
split_point

11568803

In [11]:
test = learn_df.iloc[split_point:, :]
train = learn_df.iloc[:split_point, :]

In [17]:
test_train = {'test': test,
            'train': train}

In [22]:
with open('../cleaned/1015_train_test.pkl', 'wb') as handle:
    pickle.dump(test_train, handle, protocol=pickle.HIGHEST_PROTOCOL)